### test ff
Powered by [Eleonora Priori](https://www.ecoaz.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


The trick above, useful to avoid scrolling output windows, does not work with recent jupyter versions; use settings.

In [1]:
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)



#generate random seed
repast4py.random.init(rng_seed=12345)
rng = repast4py.random.default_rng 

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, sector: int, labor: int, capital: float, capitalR: float, wage: float): 
                 #, minOrderDuration:int,\
                 #maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 #assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 #sectorialClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid

        self.sector=sector
        self.labor=labor
        self.capital=capital
        self.capitalR=capitalR
        self.wage=wage
    
        """
        self.unavailableLabor=0
        self.unavailableCapital=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        """

        
        



class Model:
    
    def __init__(self):
        
      
        runner.schedule_repeating_event(0.0,  1, self.interactingWithFirms)

        runner.schedule_stop(0)
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################

        fileName = "ff_with_class_limits.csv" #input("file name? ")
        simulationFirmsExAnteNumber = int(input("how many firms? "))
        
        self.smart_capital = 2
        while self.smart_capital != 0 and self.smart_capital != 1:
            self.smart_capital = int(input("use smart capital? (Please, use only 1/0 to express True/False values) "))
            if self.smart_capital != 0 and self.smart_capital != 1: print("Please, use only 1/0 to express True/False values")
                   
        
        #importing csv file containing info about firms 
        with open(fileName, newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',')#, quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1 #to skip the row of the headers
            k=0
            #for each record in .csv
            
            for row in firmReader:
                #print(row)
                labor = 0 
                if self.rowNumber>=0:
                        
                    sector = row[0]
                        
                    if row[4]=='': row[4]=0 # it pertains with last rows (naio sector 65)
                    for i in range(int(float(row[4]) * simulationFirmsExAnteNumber)// rankNum):
                        if row[0] == '1' or row[0] == '2' or row[0] == '3': #agri, silvi and fishing
                            randomizer = rng.uniform()
                            if randomizer <= 0.2: labor = 0
                            elif randomizer <= 0.9 : labor = 1
                            else: labor = 2
                        else:
                            if int(row[7]) == 0: row[7] = '1' #to avoid firms with 0 workers in the 0-9 class
                            labor= rng.integers(int(row[7]), int(row[8])+1) #ptptpt because integers exclude extremes                         
                            # at https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.integers.html
                            # random.Generator.integers(low, high=None,...
                            # high, if provided, one above the largest (signed) integer to be drawn from the distribution
                            
                        capital= float(row[11]) + rng.random()*(float(row[12]) - float(row[11]))
                        if self.smart_capital:
                            capital = labor * float(row[10]) # row[10] = recipe 
                            
                        capitalR = float(row[13])
                        wage = float(row[14])
                        
                        """
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectorialClass=int(self.rowNumber)
                        """
                        
                        aFirm =Firm(k, rank, sector, labor, capital, capitalR, wage) #, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                #maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                #sectorialClass)
                        context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
                self.simulationFirmsExPostNumber=k #one more, here is a count, not an id
        
    #interactingWithFirms
    def interactingWithFirms(self):

        #print(self.firmCount)

        if self.simulationFirmsExPostNumber==0:
            print("No firms created.")
        else:
            # a check
            countActualExPostFirmNumber = len(list(context.agents(agent_type=0)))
            if countActualExPostFirmNumber != self.simulationFirmsExPostNumber:
                print("DISASTER")
                quit()

            AFF_FirmNumber = 10000000
            Eu_FirmNumber = 20789413
            AFF_WorkerNumber = 9000000
            Eu_WorkerNumber = 152533008+9000000
            Eu_EmployeeCompensations = 7447036.79
            propFactor = Eu_FirmNumber / countActualExPostFirmNumber

            countWorkers     =0
            countEmployeeCompensations=0
            countAFF_Firms   =0
            countAFF_Workers =0
            for aFirm in context.agents(agent_type=0):
                countWorkers+=aFirm.labor
                countEmployeeCompensations+=aFirm.labor*aFirm.wage
                if int(aFirm.sector) <= 3: 
                    countAFF_Firms+=1
                    countAFF_Workers+=aFirm.labor

            print()
            table_data = [
                [' ',          'EU 27', 'simulation', 'simulation to'],
                [' ',          ' ',     ' ',          'EU scale'],
                ['AFF Firm Number', AFF_FirmNumber, countAFF_Firms, int(countAFF_Firms*propFactor)], 
                ['AFF Worker Number', AFF_WorkerNumber, countAFF_Workers, int(countAFF_Workers*propFactor)], 
                ['Eu Firm Number', Eu_FirmNumber, countActualExPostFirmNumber, int(countActualExPostFirmNumber*propFactor)],
                ['Eu Worker Number', Eu_WorkerNumber, countWorkers, int(countWorkers*propFactor)],
                ['Eu Empl. Compens.', Eu_EmployeeCompensations, countEmployeeCompensations/1000000, 
                                                                countEmployeeCompensations*propFactor/1000000]                                                                                                                              
            ]
            for row in table_data:
                print("{: >20} {: >20} {: >20} {: >20}".format(*row))

            print("\n AFF => Agriculture, Forestry and Fishing")
            
            """   
            countWorkers   =0
            countFirmsAgr  =0
            countWorkersAgr=0
            for aFirm in context.agents(agent_type=0):
                countWorkers+=aFirm.labor
                if int(aFirm.sector) <= 3: 
                    countFirmsAgr+=1
                    countWorkersAgr+=aFirm.labor
            print(f"with {countWorkers:,.0f} workers.")

            print(f"Subset of the firms of Agriculture, Forestry and Fishing {countFirmsAgr:,.0f} "+\
                    f"with {countWorkersAgr:,.0f} workers")

            
            #Europe 27
            #Agriculture, Forestry and Fishing => AFF
            actual_firm_number=  20789413      #20751071
            actual_AFF_firm_number=10000000
            actual_worker_number=9000000+152533008 
            actual_AFF_worker_number=9000000
            print()            
            print(f"Europe (27 countries): actual firm number {actual_firm_number:,.0f} "+\
                  f"with {actual_worker_number:,.0f} workers.")
            print(f"Subset in Agriculture, Forestry and Fishing: {actual_AFF_firm_number:,.0f} firms "+\
                  f"with {actual_AFF_worker_number:,.0f} workers.")

            expected_workers_in_simulation = actual_worker_number*countFirms/actual_firm_number
            expected_AFF_firm_number=actual_AFF_firm_number*countFirms/actual_firm_number
            expected_AFF_worker_number=actual_AFF_worker_number*countFirms/actual_firm_number
            
            print(f"Proportionally: expected number of workers in the simulation {expected_workers_in_simulation:,.0f};")
            print(f"                expected number of AFF firms in the simulation {expected_AFF_firm_number:,.0f}.")
            print(f"                expected number of AFF workers in the simulation {expected_AFF_worker_number:,.0f}.")
            print()            

            # https://ec.europa.eu/eurostat/databrowser/view/tec00001/default/table?lang=en
            print(f"Europe 27 actual GDP (millions) {16144780.0:,.0f} with TVA, current prices, "+\
                  "million euro, 2022 as the naio table.")
            print(f"Europe 27, naio table 2022 (milions) total Added value, gross {14303899.14:,.0f}")
            print(f"ratio {16144780.0/14303899.14:,.3f} where {100*(16144780.0/14303899.14-1):,.1f}% is "+\
                 "the effect of (+) TVA and (-) public contributions to production")
            print()
            
            print(f"Europe 27, naio table 2022 (millions) total Compensation of employees {7447036.79:,.0f}")
            print("Europe 27, naio table 2022 (millions) total Added value, gross - total "+\
                  f"Compensation of employees {14303899.14 - 7447036.79:,.0f}")
    
            print()
            if self.smart_capital: print("Using smart capital")
            else:                  print("Not using smart capital")

            #GDP estimation, no markup
            GDP=0
            GDPwages=0
            GDPr=0
            for aFirm in context.agents(agent_type=0):
                laborN=aFirm.labor
                GDP += aFirm.wage*aFirm.labor + 1000*aFirm.capital*aFirm.capitalR
                GDPwages += aFirm.wage*aFirm.labor
                GDPr += 1000*aFirm.capital*aFirm.capitalR

            print(f"Europe 27 GDP estimation (millions) without markup {GDP/1000000:,.0f}.")
            print(f"estimation of Compensation of employees {GDPwages/1000000:,.0f} "+
                  f"and estimation of Capital compensation {GDPr/1000000:,.0f}")

            # reporting results to the actual number of firms
            propFactor = actual_firm_number / countFirms
            print()
            print(f"Europe 27 GDP estimation (millions) without markup {propFactor*GDP/1000000:,.0f}, "+\
                  "corrected by proportion.")
            print(f"estimation of Compensation of employees {propFactor*GDPwages/1000000:,.0f} "+
                  f"and estimation of Capital compensation {propFactor*GDPr/1000000:,.0f}, "+\
                  "corrected by proportion.")
            """          
    


    #finish
    def finish(self):
        print("\nConcluded")
    
    def start(self):
        runner.execute()

In [3]:
def run():

    model = Model() 
    model.start()
    
run()

#file name: ff_with_class_limits.csv

how many firms?  10000000
use smart capital? (Please, use only 1/0 to express True/False values)  0



                                    EU 27           simulation        simulation to
                                                                           EU scale
     AFF Firm Number             10000000              4810136             10000393
   AFF Worker Number              9000000              4328574              8999213
      Eu Firm Number             20789413              9999597             20789413
    Eu Worker Number            161533008             77669920            161477712
   Eu Empl. Compens.           7447036.79    3373445.262209359    7013477.322032443

 AFF => Agriculture, Forestry and Fishing

Concluded
